In [1]:
# Install boto3
# pip install boto3

# Fetch your S3 Access and Secret keys from
# https://polygon.io/dashboard/keys

# Quickstart
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html

# Example
import boto3
from botocore.config import Config

# Initialize a session using your credentials
session = boto3.Session(
   aws_access_key_id='c088820e-e8b7-49d7-bd75-c8168a1f3a14',
   aws_secret_access_key='iASuja66NhcDdq_yQu7mhYZMzBqQdA2O',
)

# Create a client with your session and specify the endpoint
s3 = session.client(
   's3',
   endpoint_url='https://files.polygon.io',
   config=Config(signature_version='s3v4'),
)

# Initialize a paginator for listing objects
paginator = s3.get_paginator('list_objects_v2')

# Choose the appropriate prefix depending on the data you need:
# - 'global_crypto' for global cryptocurrency data
# - 'global_forex' for global forex data
# - 'us_indices' for US indices data
# - 'us_options_opra' for US options (OPRA) data
# - 'us_stocks_sip' for US stocks (SIP) data
prefix = 'us_stocks_sip'  # Example: Change this prefix to match your data need

# List objects using the selected prefix
for page in paginator.paginate(Bucket='flatfiles', Prefix=prefix):
    for obj in page['Contents']:
        print(obj['Key'])

# Copy example
# Specify the bucket name
bucket_name = 'flatfiles'

# Specify the S3 object key name
object_key = 'us_stocks_sip/minute_aggs_v1/2023/01/2023-01-31.csv.gz'

# Specify the local file name and path to save the downloaded file
# This splits the object_key string by '/' and takes the last segment as the file name
local_file_name = object_key.split('/')[-1]

# This constructs the full local file path
local_file_path = './' + local_file_name

# Download the file
s3.download_file(bucket_name, object_key, local_file_path)

us_stocks_sip/day_aggs_v1/2003/09/2003-09-10.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-11.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-12.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-15.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-16.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-17.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-18.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-19.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-22.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-23.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-24.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-25.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-26.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-29.csv.gz
us_stocks_sip/day_aggs_v1/2003/09/2003-09-30.csv.gz
us_stocks_sip/day_aggs_v1/2003/10/2003-10-01.csv.gz
us_stocks_sip/day_aggs_v1/2003/10/2003-10-02.csv.gz
us_stocks_sip/day_aggs_v1/2003/10/2003-10-03.csv.gz
us_stocks_sip/day_aggs_v1/2003/10/2003-10-06.csv.gz
us_stocks_si

load holidays:


In [2]:
from polygon import RESTClient
from polygon.rest.models import (
    MarketHoliday,
)

client = RESTClient("iASuja66NhcDdq_yQu7mhYZMzBqQdA2O")

holidays = client.get_market_holidays()

#print(holidays)

# print date, name, and exchange
for holiday in holidays:
    # verify this is an exchange
    if isinstance(holiday, MarketHoliday):
        print("{:<15}{:<15} ({})".format(holiday.date, holiday.name, holiday.exchange))

2025-05-26     Memorial Day    (NYSE)
2025-05-26     Memorial Day    (NASDAQ)
2025-06-19     Juneteenth      (NYSE)
2025-06-19     Juneteenth      (NASDAQ)
2025-07-03     Independence Day (NASDAQ)
2025-07-03     Independence Day (NYSE)
2025-07-04     Independence Day (NASDAQ)
2025-07-04     Independence Day (NYSE)
2025-09-01     Labor Day       (NASDAQ)
2025-09-01     Labor Day       (NYSE)
2025-11-27     Thanksgiving    (NYSE)
2025-11-27     Thanksgiving    (NASDAQ)
2025-11-28     Thanksgiving    (NASDAQ)
2025-11-28     Thanksgiving    (NYSE)
2025-12-24     Christmas       (NASDAQ)
2025-12-24     Christmas       (NYSE)
2025-12-25     Christmas       (NASDAQ)
2025-12-25     Christmas       (NYSE)
2026-01-01     New Years Day   (NASDAQ)
2026-01-01     New Years Day   (NYSE)
2026-01-19     Martin Luther King, Jr. Day (NYSE)
2026-01-19     Martin Luther King, Jr. Day (NASDAQ)
2026-02-16     Washington's Birthday (NYSE)
2026-02-16     Washington's Birthday (NASDAQ)
2026-04-03     Good Frid

In [12]:
import boto3
from botocore.config import Config
import pandas as pd
from io import BytesIO
from pathlib import Path
from tqdm import tqdm
import gzip
import pandas_market_calendars as mcal


def get_spx_tickers():
    df_spx_constituents = pd.read_csv("SPX_constituents_2023_filled.csv")
    df_spx_constituents = df_spx_constituents.rename(columns={"Symbol": "ticker"})
    #df_spx_constituents = df_spx_constituents[["ticker", "Name"]]
    tickers= set(df_spx_constituents["ticker"].unique())
    return tickers

# ==== SETUP ====

# AWS session (use your real keys — NEVER hardcode into production!)
session = boto3.Session(
    aws_access_key_id='c088820e-e8b7-49d7-bd75-c8168a1f3a14',
    aws_secret_access_key='iASuja66NhcDdq_yQu7mhYZMzBqQdA2O',
)

s3 = session.client(
    's3',
    endpoint_url='https://files.polygon.io',
    config=Config(signature_version='s3v4'),
)

bucket_name = 'flatfiles'
prefix = 'us_stocks_sip/minute_aggs_v1'

# Local output
output_dir = Path("./filtered_data")
output_dir.mkdir(exist_ok=True)

# Your ticker list
my_tickers = get_spx_tickers()
# Dates
#dates = pd.date_range(start="2023-01-01", end="2023-12-31")
#dates = dates[dates.dayofweek < 5]  # 0=Monday, ..., 4=Friday

nyse = mcal.get_calendar("NYSE")
schedule = nyse.schedule(start_date="2023-01-01", end_date="2023-01-10")
dates = schedule.index 

missing_files = []

# ==== PROCESS LOOP ====

for date in tqdm(dates):
    date_str = date.strftime("%Y-%m-%d")
    year = date.strftime("%Y")
    month = date.strftime("%m")
    
    # Build the object key
    object_key = f"{prefix}/{year}/{month}/{date_str}.csv.gz"
    
    try:
        # Download file into memory
        response = s3.get_object(Bucket=bucket_name, Key=object_key)
        compressed_data = response['Body'].read()
        
        # Decompress and load into pandas directly
        with gzip.GzipFile(fileobj=BytesIO(compressed_data)) as gz:
            df = pd.read_csv(gz)

    except s3.exceptions.NoSuchKey:
        #print(f"File not found for {date_str}")
        missing_files.append(date_str)
        continue
    except Exception as e:
        print(f"Failed to load {object_key}: {e}")
        continue

    # Check expected columns
    if 'ticker' not in df.columns:
        print(f"No 'ticker' column in {object_key}")
        continue

    # Filter
    filtered = df[df['ticker'].isin(my_tickers)]

    if filtered.empty:
        print(f"No tickers found for {date_str}")
        continue

    # Save
    output_file = output_dir / f"{date_str}_filtered.csv"
    filtered.to_csv(output_file, index=False)

print(f"Missing files: {missing_files}")

100%|██████████| 6/6 [00:21<00:00,  3.56s/it]

Missing files: []


In [ ]:
def get_spx_tickers():
    df_spx_constituents = pd.read_csv("SPX_constituents_2023_filled.csv")
    df_spx_constituents = df_spx_constituents.rename(columns={"Symbol": "ticker"})
    #df_spx_constituents = df_spx_constituents[["ticker", "Name"]]
    tickers= set(df_spx_constituents["ticker"].unique())
    return tickers
tickers = get_spx_tickers()
# print(tickers)

tickers

{'A',
 'AAL',
 'AAP',
 'AAPL',
 'ABBV',
 'ABC',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALK',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APTV',
 'ARE',
 'ATO',
 'ATVI',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBWI',
 'BBY',
 'BDX',
 'BEN',
 'BF.B',
 'BIIB',
 'BIO',
 'BK',
 'BKNG',
 'BKR',
 'BLK',
 'BMY',
 'BR',
 'BRK.B',
 'BRO',
 'BSX',
 'BWA',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDAY',
 'CDNS',
 'CDW',
 'CE',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMA',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COO',
 'COP',
 'COST',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTLT',
 'CTRA',
 'C

In [11]:
import pandas_market_calendars as mcal

nyse = mcal.get_calendar("NYSE")

schedule = nyse.schedule(start_date="2023-01-01", end_date="2023-01-31")

dates = schedule.index  # This is a DatetimeIndex of real trading days
dates

DatetimeIndex(['2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
               '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12',
               '2023-01-13', '2023-01-17', '2023-01-18', '2023-01-19',
               '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25',
               '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31'],
              dtype='datetime64[ns]', freq=None)

In [27]:
import pandas as pd
import numpy as np
from pathlib import Path
from zoneinfo import ZoneInfo  # Built-in from Python 3.9+

def compute_realized_volatility(input_file, resample_interval='5min'):
    df = pd.read_csv(input_file)
    date = input_file.stem.split("_")[0]

    if 'window_start' not in df.columns:
        raise ValueError("Missing 'window_start' column in file.")

    # Convert nanoseconds to datetime
    df['timestamp'] = pd.to_datetime(df['window_start'], unit='ns')

    # Localize to UTC, then convert to NY timezone (handles DST)
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(ZoneInfo('America/New_York'))
    df = df.sort_values('timestamp')  # Sort once globally

    # Normalize close price column
    if 'close' not in df.columns and 'price' in df.columns:
        df['close'] = df['price']
    elif 'close' not in df.columns:
        raise ValueError("No 'close' or 'price' column found.")

    # Determine scaling factor
    interval_minutes = int(resample_interval.replace('min', ''))
    annualization_factor = np.sqrt(252 * 24 * 60 / interval_minutes)

    vol_data = []

    for ticker, group in df.groupby('ticker',sort=False):
        group = group.set_index('timestamp')

        # Keep only regular NYSE hours (after timezone adjustment!)
        group = group.between_time('09:30', '16:00')

        if group.empty:
            continue

        price_series = group['close'].resample(resample_interval).last().dropna()

        log_returns = np.log(price_series / price_series.shift(1)).dropna()

        if len(log_returns) < 2:
            continue

        #vol = log_returns.std() * annualization_factor
        vol = np.sqrt((log_returns ** 2).mean()) * annualization_factor

        vol_data.append({
            'ticker': ticker,
            'date': date,
            f'annualized_vol_{resample_interval}': vol
        })

    return pd.DataFrame(vol_data)

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from zoneinfo import ZoneInfo  # Built-in from Python 3.9+


import sys
sys.path.append('./src')  # only needed if you're not packaging yet

from vol_utils import compute_log_returns, annualized_volatility_mean_square


def compute_realized_volatility(input_file, intervals=['5min', '15min', '30min']):
    df = pd.read_csv(input_file)
    date = input_file.stem.split("_")[0]

    if 'window_start' not in df.columns:
        raise ValueError("Missing 'window_start' column in file.")

    # Convert nanoseconds to datetime
    df['timestamp'] = pd.to_datetime(df['window_start'], unit='ns')
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(ZoneInfo('America/New_York'))
    df = df.sort_values('timestamp')

    if 'close' not in df.columns and 'price' in df.columns:
        df['close'] = df['price']
    elif 'close' not in df.columns:
        raise ValueError("No 'close' or 'price' column found.")

    vol_data = []

    for ticker, group in df.groupby('ticker', sort=False):
        group = group.set_index('timestamp')
        group = group.between_time('09:30', '15:59')

        if group.empty:
            continue

        row = {
            'ticker': ticker,
            'date': date,
            'open': group['close'].iloc[0],
            'close': group['close'].iloc[-1],
        }

        for interval in intervals:
            interval_minutes = int(interval.replace('min', ''))
            annualization_factor = np.sqrt(252 * 24 * 60 / interval_minutes)

            price_series = group['close'].resample(interval).last().dropna()
            log_returns = np.log(price_series / price_series.shift(1)).dropna()

            if len(log_returns) < 2:
                row[f'annualized_vol_{interval}'] = np.nan
            else:
                vol = np.sqrt((log_returns ** 2).mean()) * annualization_factor
                row[f'annualized_vol_{interval}'] = vol

        vol_data.append(row)

    return pd.DataFrame(vol_data)

In [7]:
def compute_realized_volatility(input_file, intervals=['5min', '15min'], estimator='mean_square'):
    df = pd.read_csv(input_file)
    date = input_file.stem.split("_")[0]

    if 'window_start' not in df.columns:
        raise ValueError("Missing 'window_start' column in file.")

    # Convert nanoseconds to datetime
    df['timestamp'] = pd.to_datetime(df['window_start'], unit='ns')
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert(ZoneInfo('America/New_York'))
    df = df.sort_values('timestamp')

    if 'close' not in df.columns and 'price' in df.columns:
        df['close'] = df['price']
    elif 'close' not in df.columns:
        raise ValueError("No 'close' or 'price' column found.")

    vol_data = []

    for ticker, group in df.groupby('ticker', sort=False):
        group = group.set_index('timestamp')
        group = group.between_time('09:30', '15:59')

        if group.empty:
            continue

        row = {
            'ticker': ticker,
            'date': date,
            'open': group['close'].iloc[0],
            'close': group['close'].iloc[-1],
        }
        for interval in intervals:
            interval_minutes = int(interval.replace('min', ''))

            price_series = group['close'].resample(interval).last().dropna()
            log_returns = compute_log_returns(price_series)

            if len(log_returns) < 2:
                row[f'annualized_vol_{interval}'] = np.nan
            else:
                if estimator == 'std':
                    vol = annualized_volatility_std(log_returns, interval_minutes)
                elif estimator == 'parkinson':
                    high = group['high'].resample(interval).max().dropna()
                    low = group['low'].resample(interval).min().dropna()
                    vol = annualized_volatility_parkinson(high, low, interval_minutes)
                else:
                    vol = annualized_volatility_mean_square(log_returns, interval_minutes)

                row[f'annualized_vol_{interval}'] = vol
        vol_data.append(row)

    return pd.DataFrame(vol_data)

In [30]:
df = compute_realized_volatility(Path("filtered_data/2023-01-03_filtered.csv"),
                                 intervals=['1min', '5min', '15min'])

print(df.head())

  ticker        date      open   close  annualized_vol_1min  \
0    AAL  2023-01-03   12.9300   12.72             0.651786   
1    CAT  2023-01-03  239.0585  238.88             0.494511   
2    DFS  2023-01-03   98.9200   96.99             0.522493   
3   META  2023-01-03  123.8000  124.74             0.783257   
4   EBAY  2023-01-03   42.2800   42.15             0.526548   

   annualized_vol_5min  annualized_vol_15min  
0             0.645646              0.659954  
1             0.448392              0.394926  
2             0.566101              0.686230  
3             0.746571              0.765842  
4             0.539219              0.586724  


In [8]:
from pathlib import Path
import pandas as pd

input_dir = Path("../data/filtered_data")
output_file = Path("./all_vols.csv")

# Collect all vol data into this list
all_results = []

# Loop over all daily filtered files
for input_file in input_dir.glob("*_filtered.csv"):
    result_df = compute_realized_volatility(input_file, intervals=["1min", "5min", "15min"])
    if not result_df.empty:
        all_results.append(result_df)

# Combine into one big DataFrame
if all_results:
    combined_df = pd.concat(all_results, ignore_index=True)
    combined_df.to_csv(output_file, index=False)
    print(f"Saved master volatility file to {output_file}")
else:
    print("No results generated.")

Saved master volatility file to all_vols.csv


In [37]:
test=pd.concat(all_results, ignore_index=True)
test[test['ticker'] == 'AAPL']

,ticker,date,open,close,annualized_vol_5min,annualized_vol_15min,annualized_vol_30min
5,AAPL,2023-05-05,171.2350,173.6200,0.379440,0.225242,0.157654
510,AAPL,2023-11-22,192.3650,191.2893,0.261761,0.211095,0.128258
1009,AAPL,2023-05-02,170.1053,168.5800,0.261900,0.234421,0.291105
1506,AAPL,2023-02-27,148.1600,148.0000,0.260656,0.263737,0.176315
2018,AAPL,2023-07-06,190.3100,191.8900,0.216829,0.176245,0.184920
...,...,...,...,...,...,...,...
122144,AAPL,2023-05-31,177.6900,177.3100,0.395858,0.401052,0.323309
122631,AAPL,2023-03-28,157.9100,157.6200,0.259128,0.272430,0.236865
123148,AAPL,2023-06-09,181.6800,181.0200,0.280218,0.277697,0.193036
123641,AAPL,2023-12-29,194.3200,192.8300,0.189351,0.183518,0.228834


In [20]:
import pandas as pd

df = pd.read_csv("filtered_data/2023-01-03_filtered.csv")

# Convert from nanoseconds
df['timestamp'] = pd.to_datetime(df['window_start'], unit='ns')

# What tickers are in this file?
#print(df['ticker'].unique())

# Inspect timestamps per ticker
for ticker in df['ticker'].unique():
    times = df[df['ticker'] == ticker]['timestamp']
    print(f"{ticker}: min={times.min()}, max={times.max()}, count={len(times)}")

A: min=2023-01-03 14:29:00, max=2023-01-03 21:03:00, count=375
AAL: min=2023-01-03 09:00:00, max=2023-01-04 00:59:00, count=496
AAP: min=2023-01-03 12:53:00, max=2023-01-04 00:57:00, count=383
AAPL: min=2023-01-03 09:00:00, max=2023-01-04 00:59:00, count=867
ABBV: min=2023-01-03 11:24:00, max=2023-01-03 23:43:00, count=403
ABC: min=2023-01-03 12:11:00, max=2023-01-03 21:02:00, count=371
ABT: min=2023-01-03 10:29:00, max=2023-01-03 21:18:00, count=396
ACGL: min=2023-01-03 14:29:00, max=2023-01-03 21:02:00, count=390
ACN: min=2023-01-03 14:30:00, max=2023-01-03 21:12:00, count=394
ADBE: min=2023-01-03 09:04:00, max=2023-01-03 23:28:00, count=415
ADI: min=2023-01-03 12:42:00, max=2023-01-03 21:53:00, count=393
ADM: min=2023-01-03 09:15:00, max=2023-01-03 23:41:00, count=399
ADP: min=2023-01-03 14:27:00, max=2023-01-03 21:02:00, count=382
ADSK: min=2023-01-03 13:01:00, max=2023-01-03 21:02:00, count=375
AEE: min=2023-01-03 14:30:00, max=2023-01-03 21:02:00, count=366
AEP: min=2023-01-03 13